In [5]:


from ucimlrepo import fetch_ucirepo
import numpy as np
# fetch dataset
heart_disease = fetch_ucirepo(id=45)

# data (as pandas dataframes)
X = heart_disease.data.features
print(X)
y = heart_disease.data.targets





     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0     63    1   1       145   233    1        2      150      0      2.3   
1     67    1   4       160   286    0        2      108      1      1.5   
2     67    1   4       120   229    0        2      129      1      2.6   
3     37    1   3       130   250    0        0      187      0      3.5   
4     41    0   2       130   204    0        2      172      0      1.4   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
298   45    1   1       110   264    0        0      132      0      1.2   
299   68    1   4       144   193    1        0      141      0      3.4   
300   57    1   4       130   131    0        0      115      1      1.2   
301   57    0   2       130   236    0        2      174      0      0.0   
302   38    1   3       138   175    0        0      173      0      0.0   

     slope   ca  thal  
0        3  0.0   6.0  
1        2  3.0   3.0  
2        2  2.0

Implementacja regresji logistycznej. Jako że atrybut num ma wartości 0 do 4 a wartości >=1 oznaczają chorobę
serca, więc przypisujemy wartości 1 do wartości >=1 a wartości 0 pozostawiamy bez zmian. Dzięki temu regresja logistyczna która zwraca wartości od 0 do 1 ma więcej sensu i będzie łatwiejsza do implementacji

In [6]:
class MyLogisticRegression:
    def __init__(self, learning_rate=0.05, num_iterations=1000, batch_size=32):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.batch_size = batch_size

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        # Inicjalizacja wag i biasu
        #initalize random weights
        self.theta = np.random.rand(X.shape[1])
        self.bias = 0

        m = X.shape[0]

        for _ in range(self.num_iterations):
            # Mieszanie danych treningowych
            indices = np.arange(m)
            np.random.shuffle(indices)

            for i in range(0, m, self.batch_size):
                batch_indices = indices[i:i + self.batch_size]
                X_batch = []
                y_batch = []
                for index in batch_indices:
                    X_batch.append((X.iloc[index]).to_numpy())
                    y_batch.append(y[index][0])
                X_batch = np.array(X_batch)


                linear_model = np.dot(X_batch, self.theta) + self.bias
                predictions = self.sigmoid(linear_model)

             # Obliczenie gradientów
                dw = (1 / self.batch_size) * np.dot(X_batch.T, (predictions - y_batch))
                db = (1 / self.batch_size) * np.sum(predictions - y_batch)

                # Aktualizacja wag i biasu
                self.theta -= self.learning_rate * dw
                self.bias -= self.learning_rate * db

    def predict(self, X):
        linear_model = np.dot(X, self.theta) + self.bias
        predictions = self.sigmoid(linear_model)
        return (predictions > 0.5).astype(int)

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

y = np.where(y >= 1, 1, y)
mean_value_thal=X['thal'].mean()
X['thal'].fillna(value=mean_value_thal,inplace=True)
mean_value_ca=X['ca'].mean()
X['ca'].fillna(value=mean_value_ca,inplace=True)




X_train, X_test, y_train, y_test = train_test_split(X, y)

model = MyLogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

preciosion = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Preciosion: {preciosion}')
print(f'Recall: {recall}')
print(f'Accuracy: {accuracy}')
print(f'F1: {f1}')




C:\Users\kubas\AppData\Local\Temp\ipykernel_5384\2407195363.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['thal'].fillna(value=mean_value_thal,inplace=True)
C:\Users\kubas\AppData\Local\Temp\ipykernel_5384\2407195363.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['ca'].fillna(value=mean_value_ca,inplace=True)
C:\Users\kubas\AppData\Local\Temp\ipykernel_5384\2407195363.py:8: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


Preciosion: 0.7692307692307693
Recall: 0.5882352941176471
Accuracy: 0.7368421052631579
F1: 0.6666666666666667


Podział danych na zbiór treningowy i testowy